In [ ]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import joblib
import os

# Load dataset into a DataFrame
df = pd.read_csv("data/movies.csv")

# Convert release_date to datetime format; handle parsing errors as NaT
df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")

# Extract release month as a numerical feature (1–12)
df["release_month"] = df["release_date"].dt.month

# Drop rows with missing values in key numerical columns we’ll use for modeling
df = df.dropna(subset=["budget", "revenue", "release_month"])

# Quick exploratory checks to understand dataset health and distribution
print("Null counts:\n", df.isnull().sum())
print("\nBasic stats:\n", df[["budget", "revenue", "rating"]].describe())
print("\nLanguage counts:\n", df["language"].value_counts())
print("\nRelease month counts:\n", df["release_month"].value_counts())

# Show correlation between numeric features to detect multicollinearity or strong drivers
plt.figure(figsize=(8, 6))
sns.heatmap(df[["budget", "revenue", "rating"]].corr(), annot=True, cmap="coolwarm")
plt.title("Feature Correlation")
plt.show()

# Create binary target label: is_hit = whether a movie earns significantly more than its budget
# Use fallback logic: progressively test thresholds until both hit and non-hit classes exist
thresholds = [2.0, 1.5, 1.0, 0.8]  # Success defined as 2x–0.8x budget return
success = False

for t in thresholds:
    df["is_hit"] = df["revenue"] > (t * df["budget"])
    class_counts = df["is_hit"].value_counts()

    # Break as soon as we get both positive and negative classes for training
    if len(class_counts) >= 2:
        print(f"Using threshold: revenue > {t} * budget")
        success = True
        break
    else:
        print(f"Only one class at threshold {t}x: {class_counts.to_dict()}")

# If no threshold gives us diverse classes, stop the script to avoid training on a single class
if not success:
    raise ValueError("No class diversity found after testing all thresholds.")

# Select input features and target variable
X = df[["budget", "release_month"]]
y = df["is_hit"]

# Train-test split for model validation (20% test set)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features for better convergence in logistic regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train logistic regression to classify movies as hits vs non-hits
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

# Save trained model and scaler for later use (e.g., Streamlit app or API)
os.makedirs("ml", exist_ok=True)
joblib.dump((model, scaler), "ml/hit_predictor.pkl")
print("Model saved to 'ml/hit_predictor.pkl'")

# Evaluate model on the test set
y_pred = model.predict(X_test_scaled)
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

# Display confusion matrix to visualize performance across hit/non-hit predictions
ConfusionMatrixDisplay.from_estimator(model, X_test_scaled, y_test)
plt.title("Confusion Matrix")
plt.show()